In [2]:
import numpy as np

# Функция перестановки строк для приближения диагонального преобладания
def reorder_matrix_for_dominance(matrix, rhs):
    size = matrix.shape[0]
    A_mod = np.zeros_like(matrix)
    b_mod = np.zeros_like(rhs)
    available = list(range(size))

    for i in range(size):
        dominant_idx = max(available, key=lambda r: abs(matrix[r, i]))
        A_mod[i] = matrix[dominant_idx]
        b_mod[i] = rhs[dominant_idx]
        available.remove(dominant_idx)

    for i in range(size):
        row_total = np.sum(np.abs(A_mod[i])) - abs(A_mod[i, i])
        if abs(A_mod[i, i]) <= row_total:
            print("Внимание: диагональное преобладание не достигнуто.")

    return A_mod, b_mod

# Итерационный метод Якоби
def jacobi_solver(A_init, b_init, tol=1e-10, max_steps=1000, force_diag=True):
    A, b = reorder_matrix_for_dominance(A_init, b_init) if force_diag else (A_init, b_init)
    dim = len(b)
    current = np.zeros(dim)

    for step in range(max_steps):
        next_iter = np.zeros(dim)
        for i in range(dim):
            terms = b[i] - np.dot(A[i, :], current) + A[i, i] * current[i]
            next_iter[i] = terms / A[i, i]
        if np.linalg.norm(next_iter - current, ord=2) < tol:
            return next_iter, step + 1
        current = next_iter
    print("Достигнут лимит по итерациям.")
    return current, max_steps

# Метод Зейделя (последовательная итерация)
def gauss_seidel(A_start, b_start, eps=1e-10, max_loops=1000, adjust=True):
    A, b = reorder_matrix_for_dominance(A_start, b_start) if adjust else (A_start, b_start)
    n = len(b)
    x_vec = np.zeros(n)

    for cycle in range(max_loops):
        x_old = x_vec.copy()
        for i in range(n):
            before = np.dot(A[i, :i], x_vec[:i])
            after = np.dot(A[i, i + 1:], x_vec[i + 1:])
            x_vec[i] = (b[i] - before - after) / A[i, i]
        if np.linalg.norm(x_vec - x_old, ord=2) < eps:
            return x_vec, cycle + 1

    print("Превышено допустимое число шагов.")
    return x_vec, max_loops

# Входные данные
A_input = np.array([[2, 2, 10],
                    [10, 1, 1],
                    [2, 10, 1]])
b_input = np.array([14, 12, 13])

print("Матрица A:")
print(A_input)
print("Вектор b:", b_input)

solution1, iters1 = jacobi_solver(A_input, b_input)
print("\nРезультат (Якоби):", solution1)
print("Итераций:", iters1)
print("Ax - b:", np.dot(A_input, solution1) - b_input)

solution2, iters2 = gauss_seidel(A_input, b_input)
print("\nРезультат (Зейдель):", solution2)
print("Итераций:", iters2)
print("Ax:", np.dot(A_input, solution2))

Матрица A:
[[ 2  2 10]
 [10  1  1]
 [ 2 10  1]]
Вектор b: [14 12 13]

Результат (Якоби): [1. 1. 1.]
Итераций: 21
Ax - b: [5.75468562e-11 3.62518904e-11 4.56754634e-11]

Результат (Зейдель): [1. 1. 1.]
Итераций: 9
Ax: [14. 12. 13.]
